# From Mantid to Scipp
## Data types
### Workspaces

| Mantid | Scipp |
| ---| --- |
| `Workspace2D` | `DataArray` |
| `EventWorkspace` | `DataArray` |
| `WorkspaceSingleValue` | `DataArray` |
| `MDHistoWorkspace` | `DataArray` |
| `MDEventWorkspace` | not supported |
| `TableWorkspace` | `Dataset` |
| `WorkspaceGroup` | `Dataset` (aligned dimensions), otherwise Python `list` or `dict` |

#### Notes
- In many cases it may be desirable to use `Dataset` instead of `DataArray`. You can easily create a `Dataset` directly from a `DataArray`.
- Scipp takes basic geometric information from Mantid's instrument in the form of positions. Detector grouping by spectrum is respected. Upon conversion, Scipp will perform spherical coordinate averaging for the group based on the beam direction, this preserves the average scattering angle between a group of detectors and the spectra respresenting the group. This may yield slightly different detector and spectrum positions between what is natively stored in Mantid's instrument and Scipp.
- Run and Sample are copied over to scipp from any MatrixWorkspace derived Workspaces.
- Scipp (or rather conversion to scipp) is currently still incomplete and does not carry over all information from a workspace.

### Other

| Mantid | Scipp |
| ---| --- |
| `DetectorInfo` | `Dataset` |

## Concepts

Mantid's `MatrixWorkspace` (the common base class of `Workspace2D` and `EventWorkspace`) uses the terms "X", "Y", and "E" to refer to one of its axes, the data values, and the uncertainties.

- Mantid stores **standard-deviations** in "E", whereas scipp stores **variances**.
- Typically Mantid's "X" is the coordinate axis for the time-of-flight dimension, or the dimension derived from it.
- Mantid's "Y" is not the axis for the second dimension, but the **data**.
- Mantid's "X", "Y", and "E" are 1-D arrays of 1-D arrays, whereas scipp stores 2-D (or higher) arrays, if applicable.

We have the following "equivalence":

| Mantid | Scipp | comment |
| ---| --- | --- |
| `ws.readY(i)` | `data.values` |
| `ws.readE(i)` | `data.variances` | square former, or `sqrt` latter |
| `ws.readX(i)` | `data.coords['tof'].values` | dimension label may vary |
| `ws.getAxis(0).getUnit()` | `data.coords['tof'].unit` | dimension label may vary |
| `ws.getAxis(1)` | `data.coords['spectrum']` | dimension label may vary |

Here `i` is the index along the second axis (axis index `1`).
Mantid's `readX`, `readY`, and `readE` always return 1-D arrays, whereas the `values` and `variances` properties in scipp return a multi-dimensional array.
That is, there is no actual equivalence.


## Algorithms

### Notes
- In **Mantid** a Python variable referencing **a workspace is** under the hood **a global variable**.
  Unless specified otherwise the variable name is the name of the workspace in the [AnalysisDataService](https://docs.mantidproject.org/nightly/concepts/AnalysisDataService.html).
  For marginally more clarity, the examples in the following therefore use the string-based syntax for specifying output workspaces.
  *In scipp there is no such limitation and everything behaves just like normal variables in Python.*
- Unless stated otherwise, the following code examples assume datasets or data arrays have `'tof'` for what Mantid calls "X" and `'spectrum'` why Mantid calls "Y" or "spectrum axis".
- There is no strict 1:1 equivalence between Mantid workspaces and functionality in scipp.
  The examples below give the most common examples, but in many cases exceptions apply and detailed behavior may differ.
  If in doubt, consult the Mantid algorithm documentation and the scipp documentation.

In [ ]:
import mantid.simpleapi as mantid
import scipp as sc
import numpy as np

### Generic algorithms

#### CloneWorkspace

In [ ]:
mantid.CloneWorkspace(InputWorkspace='data', OutputWorkspace='copy')

Equivalent in scipp:

In [ ]:
copy = data.copy()

#### DeleteWorkspace

In [ ]:
mantid.DeleteWorkspace(Workspace='data')

Equivalent in scipp:

In [ ]:
del data

#### ExtractSingleSpectrum

In [ ]:
mantid.ExtractSingleSpectrum(
    InputWorkspace='data',
    OutputWorkspace='spec',
    WorkspaceIndex=7)

Equivalent in scipp:

In [ ]:
spec = data['spectrum', 7]

If an actual *copy* is required use:

In [ ]:
spec = data['spectrum', 7].copy()

#### ExtractSpectra / CropWorkspace

In [ ]:
mantid.ExtractSpectra(
    InputWorkspace='data',
    OutputWorkspace='spectra',
    StartWorkspaceIndex=7,
    EndWorkspaceIndex=88)

Equivalent in scipp:

In [ ]:
spectra = data['spectrum', 7:89]

If an actual *copy* is required use:

In [ ]:
spectra = data['spectrum', 7:89].copy()

#### Transpose

In [ ]:
mantid.Transpose(InputWorkspace='data', OutputWorkspace='data')

Equivalent in scipp:
Transposing is *implicit* and automatic based on dimension labels and not required for any of the common operations, including plotting.

#### AppendSpectra

In [ ]:
mantid.AppendSpectra(
    InputWorkspace1='data1',
    InputWorkspace2='data2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, 'spectrum')

#### ConjoinXRuns

In [ ]:
mantid.ConjoinXRuns(
    InputWorkspaces='data1,data2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, 'tof')

#### ConjoinSpectra

In [ ]:
mantid.ConjoinSpectra(
    InputWorkspaces='ws1,ws2',
    OutWorkspace='spec_vs_temperature'
    WorkspaceIndex=7,
    LabelUsing="temperature",
    LabelValue="Mean")

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, 'temperature')
sample_temperature = data.attrs['sample'].value['temperature']
data.coords['temperature'] = sc.mean(sample_temperature, 'time')
spec_vs_temperature = data['temperature', 7]

#### GroupWorkspaces

In [ ]:
mantid.GroupWorkspaces(
    InputWorkspaces='ws1,ws2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.Dataset({
    'data1':data_array1,
    'data2':data_array2})

This requires aligned dimensions (matching coordinates) in all input arrays.
Use a Python `dict` or `list` for grouping unaligned data.

#### Rebin  `Workspace2D` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace='histo',
    OutputWorkspace='histo',
    Params='0,100,20000')

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
histo = sc.rebin(histo, 'tof', edges)

#### Rebin  `EventWorkspace` preserving events

In [ ]:
mantid.Rebin(
    InputWorkspace='events',
    OutputWorkspace='events',
    Params='0,100,20000',
    PreserveEvents=True)

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
events.coords['tof'] = edges

#### Rebin  `EventWorkspace` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace='events',
    OutputWorkspace='histo',
    Params='0,100,20000',
    PreserveEvents=False)

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
histo = sc.histogram(events, edges)

#### Rebin with logarithmic bins

In [ ]:
mantid.Rebin(
    InputWorkspace='histo',
    OutputWorkspace='histo',
    Params='2,-0.035,10')

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    ['tof'],
    values=np.geomspace(start=2, stop=10, num=100),
    unit=sc.units.us)
histo = sc.rebin(histo, 'tof', edges)

Bin edges in scipp can be created from an arbitrary array with increasing values, the use of `numpy.geomspace` is simply one example for generating bins spaced evenly on a log scale.

#### Scale (multiplication)

In [ ]:
mantid.Scale(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
data *= 7.5

#### Scale (addition)

In [ ]:
mantid.Scale(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Add")

Equivalent in scipp:

In [ ]:
data += 7.5

If the data is not dimensionless, the correct unit must be specified:

In [ ]:
data += 7.5 * sc.units.us

#### ScaleX

In [ ]:
mantid.ScaleX(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
data.coords['tof'] *= 7.5

#### SumSpectra

In [ ]:
mantid.SumSpectra(
    InputWorkspace='data',
    OutputWorkspace='summed',
    StartWorkspaceIndex=7,
    EndWorkspaceIndex=88)

Equivalent in scipp:

In [ ]:
summed = sc.sum(data['spectrum', 7:89], 'spectrum')

### Neutron-scattering specific algorithms
#### ConvertUnits

In [ ]:
ConvertUnits(InputWorkspace='tof', OutputWorkspace='dspacing', Target='dSpacing')

Equivalent in scipp:

In [ ]:
dspacing = sc.neutron.convert(tof, 'tof', 'd-spacing')

Note that scipp has no equivalent to the `EMode` and `EFixed` settings of `ConvertUnits`.
Instead, this information is read from the input data, if available (note that currently only elastic scattering is supported).